<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# Tutorial: *Exploring the results from an ELM ensemble simulation*
This tutorial illustrates how to evaluate the results from a modest parameter ensemble perturbation experiment run at one of the four [NGEE-Arctic](https://ngee-arctic.ornl.gov/) study sites.


#### ------------------------------------------------------------------------------------------------------------

<br>

#### To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

Note: This notebook also outputs a png and pdf copy of each figure in the "/work/figures" directory

In [ ]:
# Load the required Python libraries

import matplotlib.pyplot as plt
import xarray
from netCDF4 import Dataset 
import os
import cftime
import ipywidgets as widgets
import glob, numpy
# create a figure folder in elmwork
figdir = os.path.expanduser('~')+'/work/figures'
if not os.path.exists(figdir):
    os.makedirs(figdir)
savefigs = True # True/False

<br>

#### This step identifies all of the previous ensemble ELM cases that are stored in your output location then creates a list to select the output by site

In [ ]:
# Create ensemble case root run output directories

output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/UQ/'
cases=numpy.asarray(glob.glob("%s*20TR*" % output_rootdir))
cases=[x.split('/')[-1] for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

<br>

#### Here we create the dropdown menu of ELM output options availible in your output volume

In [ ]:
# Show a dropdown menu to select specific case output
# Any cases that have been run in our elmoutput directory will be displayed
# Case names will contain the site codes:
# AK-BEO; AK-CLG; AK-K64G AK-TLG

display(cases_dropdown)

<br>

#### This step identifies all of the auto-generated ensemble case sub-directories

In [ ]:
## show ensemble sub-folders for the root transient case
ensemble_root_dirs=output_rootdir+cases_dropdown.value+"/"
ens_dirs=sorted(numpy.asarray(glob.glob("%s*g?*" % ensemble_root_dirs)))
ens_dirs= [ x for x in ens_dirs if "param_list" not in x ]
ens_cases=sorted([x.split('/')[-1] for x in ens_dirs])
print(''.join(['Ensemble case sub-directories: ',str(ens_cases)]))

<br>

#### Here we are iterating over each ensemble output path and storing the simulation results (in the file ELM_output.nc) as objects called "output_0, output_1, output_2, ... output_X" We will use these to plot the simulation results across ensemble members below

In [ ]:
## get model outputs
for x in range(0,len(ens_dirs),1):
  print(ens_dirs[x])
  temdir=ens_dirs[x]
  print(''.join(["output_",str(x)]))
  output_file=ens_dirs[x]+'/'+'ELM_output.nc'
  globals()[''.join(["output_",str(x)])]=xarray.open_dataset(output_file).squeeze()

<br>

#### Similar to other example notebooks, this step is just showing how to get a quick "look inside" one of the output objects. In this case its the first ensemble member

In [ ]:
print(output_0)

<br>

#### Here we create a matrix of random numbers (ensemble length, 3) to create random RGB color values that will be used in the plotting code below 

In [ ]:
# Lets create a random vector of plot colors to use in each ensemble plot below
plot_colors = numpy.random.rand(len(ens_dirs),3)

<br>

#### Use this to set the subset range for all plots below.  

**Note:** Individual plots can be edited by entering in custom "timerange=slice('XXXX-XX-XX','XXXX-XX-XX')" values

In [ ]:
# Subset output to the 1850-1855 period by default. This can be changed by the user
usrtimeslice=slice('1850-01-01','1855-01-01')

<br>

#### In the default ensemble example, the transient case is only run for the 1850-1855 period. This step plots the Net Ecosystem Exchange (NEE) variance for that period.  

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='NEE'
# Unit scaler
scaler=24*3600
# Unit label for plot
units='C flux (g C m$^{-2}$ day$^{-1}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in ecosystem Heterotrophic Respiration (HR) rate

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='HR'
# Unit scaler
scaler=24*3600
# Unit label for plot
units='C flux (g C m$^{-2}$ day$^{-1}$)'


# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in ecosystem Gross Primary Productivity (GPP)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='GPP'
# Unit scaler
scaler=24*3600
# Unit label for plot
units='C flux (g C m$^{-2}$ day$^{-1}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in ecosystem Net Primary Productivity (NPP)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='NPP'
# Unit scaler
scaler=24*3600
# Unit label for plot
units='C flux (g C m$^{-2}$ day$^{-1}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in total ecosystem carbon

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='TOTVEGC'
# Unit scaler
scaler=1
# Unit label for plot
units='Carbon stock (g C m$^{-2}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in total ecosystem nitrogen (nitrogen pool)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='TOTVEGN'
# Unit scaler
scaler=1
# Unit label for plot
units='Nitrogen stock (g N m$^{-2}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in Leaf Area Index (LAI)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='TLAI'
# Unit scaler
scaler=1
# Unit label for plot
units='Leaf Area Index (m$^{2}$ m$^{-2}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in vegetation transpiration (QVEGT)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='QVEGT'
# Unit scaler
scaler=1
# Unit label for plot
units='Canopy Transpiration (mm s$^{-1}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in total latent heat flux (EFLX_LH_TOT)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='EFLX_LH_TOT'
# Unit scaler
scaler=1
# Unit label for plot
units='LH (W m$^{-2}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()

<br>

#### Here we plot the ensemble variance in vegetation sensible heat flux (FSH_V)

**Note:** If you modified the run period you can also edit the plot below by changing the "timerange" variable. e.g. you could change to "timerange=slice('2000-01-01','2010-12-31')" if your transient run contains the years 2000-2010

In [ ]:
# Figure options
# Subset output to the 1850-1855 period by default. This can be changed by the user
timerange=usrtimeslice
# variable name
invar='FSH_V'
# Unit scaler
scaler=1
# Unit label for plot
units='SH (W m$^{-2}$)'

# Create the figure
fig, ax = plt.subplots(clear=True, figsize=(16,7))
for x in range(0,len(ens_dirs),1):
    outputnum=''.join(["output_",str(x),str([invar])])
    plotdata=eval(outputnum)
    plotdata=plotdata*scaler  
    plotlab=''.join([invar,' ens: ',str(x)])
    plotdata.sel(time=timerange).plot(ax=ax,linestyle='-',linewidth=1.3, color=plot_colors[x], label=plotlab)
    ax.legend(fontsize="small")
    ax.set_xlabel('Year', size=20)
    ax.set_ylabel(units, size=20)
if savefigs:
    plt.savefig(os.path.join(figdir,'ens'+invar+'.pdf'))
    plt.savefig(os.path.join(figdir,'ens'+invar+'.png'))
plt.show()